## Planning-level feasibility screening (pandapower)

This notebook demonstrates a **balanced, positive-sequence / RMS** power-flow screening model for a data center with an aggregated load and a PQ-controlled BESS inverter.

### Scope (what this notebook does)
- Builds a simple grid → transformer → MV bus topology
- Represents the data center as an aggregated **PQ** load with configurable power factor
- Represents the BESS inverter as a **PQ injection** (`sgen`) with explicit P/Q capability bounds (not enforced by power flow)
- Runs single-step power flow and an hourly loop

### Out of scope (by design)
- EMT / inverter firmware / protection / harmonics
- Unbalance and negative-sequence effects

The intent is **utility-style feasibility screening**, not an interconnection approval study.

In [ ]:
import pandas as pd

from network import BaseSystemSpec, build_base_network, run_powerflow, summarize_base_results

spec = BaseSystemSpec()
net = build_base_network(spec, bess_p_mw=0.0, bess_q_mvar=0.0)
run_powerflow(net)
summary = summarize_base_results(net)

print("=== Bus voltages (pu) ===")
display(summary["bus"])

print("=== Transformer loading (%) ===")
display(summary["trafo"][["loading_percent"]])

print("=== Grid import (MW / MVAr) ===")
display(summary["grid"])

In [ ]:
from dispatch import DispatchLimits
from scenarios import DEFAULT_ARCHITECTURES
from run_simulation import SimulationConfig, run_hourly_simulation, plot_results

arch = DEFAULT_ARCHITECTURES["ac_racks"]
sim = SimulationConfig(hours=24)

limits = DispatchLimits(
    bess_p_min_mw=spec.bess_p_min_mw,
    bess_p_max_mw=spec.bess_p_max_mw,
    bess_q_min_mvar=arch.bess_q_min_mvar,
    bess_q_max_mvar=arch.bess_q_max_mvar,
)

df = run_hourly_simulation(
    system=spec,
    arch=arch,
    sim=sim,
    dispatch_limits=limits,
    seed=42,
    mode="mixed",
)

display(df.head())
plot_results(df, out_prefix=None)

In [ ]:
# PF sensitivity example (screening)

arch = DEFAULT_ARCHITECTURES["dc_800v"]
sim = SimulationConfig(hours=24)

limits = DispatchLimits(
    bess_p_min_mw=spec.bess_p_min_mw,
    bess_p_max_mw=spec.bess_p_max_mw,
    bess_q_min_mvar=arch.bess_q_min_mvar,
    bess_q_max_mvar=arch.bess_q_max_mvar,
)

cases = {}
for pf in [0.94, 0.97, 0.995]:
    df_pf = run_hourly_simulation(
        system=spec,
        arch=arch,
        sim=sim,
        dispatch_limits=limits,
        seed=42,
        mode="mixed",
        pf_override=pf,
    )
    cases[pf] = df_pf

summary = pd.DataFrame(
    {
        pf: {
            "min_vm_pu": df_pf["vm_pu_dc"].min(),
            "max_vm_pu": df_pf["vm_pu_dc"].max(),
            "max_trafo_loading_pct": df_pf["trafo_loading_pct"].max(),
            "max_grid_q_mvar": df_pf["grid_q_mvar"].max(),
        }
        for pf, df_pf in cases.items()
    }
).T

display(summary)